In [23]:
from pyforest import *
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv('/Users/jianlinzeng/Downloads/train_dataset.txt')
test = pd.read_csv('/Users/jianlinzeng/Downloads/test_dataset.txt')
data = pd.concat([df,test], axis=0)

In [3]:
data.head()

,信用分,当月旅游资讯类应用使用次数,当月是否体育场馆消费,当月是否到过福州山姆会员店,当月是否景点游览,当月是否看电影,当月是否逛过福州仓山万达,当月火车类应用使用次数,当月物流快递类应用使用次数,当月网购类应用使用次数,当月视频播放类应用使用次数,当月通话交往圈人数,当月金融理财类应用使用总次数,当月飞机类应用使用次数,是否4G不健康客户,是否大学生客户,是否经常逛商场的人,是否黑名单客户,用户实名制是否通过核实,用户年龄,用户当月账户余额（元）,用户最近一次缴费距今时长（月）,用户编码,用户网龄（月）,用户话费敏感度,用户账单当月总费用（元）,用户近6个月平均消费值（元）,缴费用户当前是否欠费缴费,缴费用户最近一次缴费金额（元）,近三个月月均商场出现次数
0,664.0,30,1,0,1,0,0,0,0,713,7145,83,2740,0,0,0,1,0,1,44,180,1,a4651f98c82948b186bdcdc8108381b4,186,3,159.20,163.86,0,99.80,75
1,530.0,0,0,0,0,0,0,0,0,414,44862,21,2731,0,1,0,1,0,1,18,110,1,aeb10247db4e4d67b2550bbc42ff9827,5,3,145.10,153.28,0,29.94,16
2,643.0,1,0,0,0,0,0,0,0,3391,4804,59,0,0,0,0,0,0,1,47,70,1,5af23a1e0e77410abb25e9a7eee510aa,145,1,120.20,109.64,0,49.90,1
3,649.0,5,1,0,1,0,0,0,0,500,3141,78,1931,0,0,0,1,0,1,55,90,1,43c64379d3c24a15b8478851b22049e4,234,3,167.42,92.97,0,99.80,26
4,648.0,0,0,0,1,0,0,0,0,522,59,70,64,0,0,0,1,0,1,40,80,1,f1687f3b8a6f4910bd0b13eb634056e2,76,3,101.00,95.47,0,49.90,44


In [4]:
bool_feature = ['当月是否体育场馆消费', '当月是否景点游览', '当月是否看电影', '当月是否到过福州山姆会员店', '当月是否逛过福州仓山万达',
                       '缴费用户当前是否欠费缴费', '是否经常逛商场的人', '是否大学生客户', '是否4G不健康客户', '是否黑名单客户',
                       '用户最近一次缴费距今时长（月）', '用户实名制是否通过核实']
num_feature = ['用户话费敏感度', '用户年龄', '近三个月月均商场出现次数', '当月火车类应用使用次数', '当月飞机类应用使用次数',
                      '当月物流快递类应用使用次数', '用户当月账户余额（元）', '用户网龄（月）', '缴费用户最近一次缴费金额（元）',
                      '当月通话交往圈人数', '当月旅游资讯类应用使用次数', '当月金融理财类应用使用总次数', '当月网购类应用使用次数',
                      '当月视频播放类应用使用次数', '用户账单当月总费用（元）', '用户近6个月平均消费值（元）']
#这里我们需要根据特征的分布 将特征划分为连续特征 和布尔特征

In [5]:
from matplotlib import font_manager
my_font = font_manager.FontProperties(fname="/Library/Fonts/Songti.ttc")
fig = plt.figure(figsize = (15,20))
for i in range(len(num_feature)):
    fig.add_subplot(5,4,i+1)
    data[num_feature[i]].plot(kind='box')
    plt.title(num_feature[i], fontproperties = my_font)
plt.show()
#然后我们根据数值型变量的分布 做进一步的调整

<Figure size 1500x2000 with 16 Axes>

In [6]:
data.loc[df['用户年龄'] == 0, '用户年龄'] = None 
data.loc[df['用户年龄'] > 100, '用户年龄'] = None
data.loc[df['用户话费敏感度'] == 0, '用户话费敏感度'] = None
data.loc[df['用户近6个月平均消费值（元）'] == 0, '用户近6个月平均消费值（元）'] = None

data.rename(columns={'用户编码': 'id', '信用分': 'score'}, inplace=True)
#根据分布将部分脱密填为0的值 还原成none 方便后续模型识别

In [7]:
#特征衍生
data['前五个月总和'] = data['用户近6个月平均消费值（元）'] * data['用户网龄（月）'].apply(lambda x: min(x, 6)) - data['用户账单当月总费用（元）']
data['当月费用偏差'] = data['用户账单当月总费用（元）'] - data['用户近6个月平均消费值（元）']
data['最近缴费比例'] = data['缴费用户最近一次缴费金额（元）'] / data['用户账单当月总费用（元）']

data['次数'] = data[['当月网购类应用使用次数', '当月物流快递类应用使用次数', '当月金融理财类应用使用总次数',
                   '当月视频播放类应用使用次数', '当月飞机类应用使用次数', '当月火车类应用使用次数', '当月旅游资讯类应用使用次数']].sum(axis=1)

for col in ['当月金融理财类应用使用总次数', '当月旅游资讯类应用使用次数']:  # 这两个比较积极向上一点                                                   
    data[col + '_百分比'] = data[col] / data['次数']
data['注册月份'] = data['用户网龄（月）'] % 12

In [27]:
feature_name = [x for x in data.columns if x!='id' and x!='score']
train = data[:df.shape[0]]
test = data[df.shape[0]:]
x_train = train[feature_name].values
y_train = train['score'].values

In [28]:
#使用lightgbm模型开始调参, 这里我们使用群友在群里分享的最佳参数
param = {'num_leaves': 48,
         'min_data_in_leaf': 50,
         'objective': 'regression_l1',
         'max_depth': 6,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.55,
         "bagging_freq": 1,
         "bagging_fraction": 0.8,
         "seed": 8888,
         "metric": 'mae',
         "lambda_l1": 0.5,
         "lambda_l2": 5,
         "verbosity": -1}


In [36]:
folds = KFold(n_splits=5, shuffle=True, random_state=0)
splits = folds.split(x_train, y_train)
predictions_lgb = 0
lgb_error = {}
for fold_, (trn_idx, val_idx) in enumerate(splits):
    print("fold n°{}".format(fold_ + 1))
    trn_data = lgb.Dataset(x_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(x_train[val_idx], y_train[val_idx])

    num_round = 20000
    clf = lgb.train(param, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=100,
                    early_stopping_rounds=100)
    lgb_error[fold_] = clf.predict(train[feature_name], num_iteration=clf.best_iteration)
    predictions_lgb += clf.predict(test[feature_name], num_iteration=clf.best_iteration) / 5
    
    
#这里我们使用折叠后的数据来模型的训练，然后我们选训练后模型对验证集进行预测，取他们最终的平均数来做我们最后的结果

fold n°1
Training until validation scores don't improve for 100 rounds.
[100]	training's l1: 20.3155	valid_1's l1: 20.4039
[200]	training's l1: 16.287	valid_1's l1: 16.6143
[300]	training's l1: 14.9911	valid_1's l1: 15.481
[400]	training's l1: 14.4827	valid_1's l1: 15.0708
[500]	training's l1: 14.2417	valid_1's l1: 14.9184
[600]	training's l1: 14.0953	valid_1's l1: 14.8526
[700]	training's l1: 13.9848	valid_1's l1: 14.8066
[800]	training's l1: 13.887	valid_1's l1: 14.7802
[900]	training's l1: 13.7956	valid_1's l1: 14.7619
[1000]	training's l1: 13.7171	valid_1's l1: 14.7498
[1100]	training's l1: 13.6406	valid_1's l1: 14.739
[1200]	training's l1: 13.5737	valid_1's l1: 14.7321
[1300]	training's l1: 13.5136	valid_1's l1: 14.7254
[1400]	training's l1: 13.4546	valid_1's l1: 14.722
[1500]	training's l1: 13.4013	valid_1's l1: 14.7193
[1600]	training's l1: 13.3497	valid_1's l1: 14.7158
[1700]	training's l1: 13.2952	valid_1's l1: 14.7123
[1800]	training's l1: 13.2437	valid_1's l1: 14.7108
[1900]

In [49]:
from sklearn.metrics import mean_absolute_error
y_predict = pd.DataFrame(lgb_error).mean(axis=1)
predictions_lgb
print("MAE CV score: {:<8.8f}".format(1/(mean_absolute_error(y_predict, y_train)+1)))
#最后我们用1/（1+MAE）来得到我们线上的预测结果的分数 

MAE CV score: 0.06960930
